In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import os, pickle
from time import time
FOLDER_PATH = ''

In [2]:
order = pd.read_csv(os.path.join(FOLDER_PATH,'data/orders.csv'))
order_train = order[order.eval_set=='train']
order_prior = order[order.eval_set=='prior']

order_product_train = pd.read_csv(os.path.join(FOLDER_PATH,'data/order_products__train.csv'))
order_product_prior = pd.read_csv(os.path.join(FOLDER_PATH,'data/order_products__prior.csv'))

In [3]:
# get validation set

validation_order_num = order_prior.groupby('user_id',as_index=False).max()[['user_id','order_number']]
validation_order = pd.merge(order_prior,validation_order_num, on=['user_id','order_number'])[['user_id','order_id']]
validation_set = pd.merge(order_product_prior,validation_order, on=['order_id'])[['user_id','product_id','reordered','order_id']]

train_order = order_prior[~order_prior.order_id.isin(validation_order.order_id)]
train_set = pd.merge(order_product_prior,train_order, on=['order_id'])[['user_id','product_id','reordered','order_id']]

test_set = pd.merge(order_product_train,order, on=['order_id'])[['user_id','product_id','reordered','order_id']]

In [5]:
pickle.dump(train_set, open(os.path.join(FOLDER_PATH,'pickle','train_set.p'),'wb'))
pickle.dump(validation_set, open(os.path.join(FOLDER_PATH,'pickle','validation_set.p'),'wb'))

pickle.dump(train_order, open(os.path.join(FOLDER_PATH,'pickle','train_order.p'),'wb'))
pickle.dump(validation_order, open(os.path.join(FOLDER_PATH,'pickle','validation_order.p'),'wb'))

pickle.dump(test_set, open(os.path.join(FOLDER_PATH,'pickle','test_set.p'),'wb'))

In [4]:
# get id mappings

order_prior_set = pd.concat([train_set,validation_set])

all_user_id = order_prior_set.user_id.unique()
all_product_id = order_prior_set.product_id.unique()

user_mapping = {all_user_id[i]:i for i in range(len(all_user_id))}
product_mapping = {all_product_id[i]:i for i in range(len(all_product_id))}

In [5]:
pickle.dump(user_mapping, open(os.path.join(FOLDER_PATH,'pickle','user_mapping.p'),'wb'))
pickle.dump(product_mapping, open(os.path.join(FOLDER_PATH,'pickle','product_mapping.p'),'wb'))